In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_values

from board_utils import *

In [2]:
assets_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
single_path = os.path.join(assets_path, 'caissa_ksample.pgn')

games = load_pgns(single_path, 1000)
games = [game for game in games if len(str(game.mainline_moves())) > 0]

Loading games: 100%|██████████| 1000/1000 [00:03<00:00, 250.41game/s]


In [3]:
def get_keys_func(board_vec):
    self_material_count = int(sum(board_vec[780:785] * [0,6,6,10,9]))
    opp_material_count = int(sum(board_vec[785:790] * [0,6,6,10,9]))
    diff_material_count = self_material_count - opp_material_count
    self_castle_available = int(sum(board_vec[768:770]) > 0)
    opp_castle_available = int(sum(board_vec[770:772]) > 0)
    position_in_endgame = int(sum(board_vec[780:790] * [0,2,2,2,1,0,2,2,2,1]) < 6)
    return self_material_count, opp_material_count, diff_material_count, self_castle_available, opp_castle_available, position_in_endgame

In [4]:
conn = psycopg2.connect(
    dbname="capstone",
    user="postgres",
    password="postgres",
)

cur = conn.cursor()

In [5]:
create_table_positions = """
CREATE TABLE IF NOT EXISTS positions (
    position_id SERIAL,
    game_id INT,
    self_material_count INT,
    opp_material_count INT,
    diff_material_count INT,
    self_castle_available INT,
    opp_castle_available INT,
    position_in_endgame INT,
    vector_int INT[],
    vector_float REAL[],
    PRIMARY KEY(position_id)
)
"""

create_table_players = """
CREATE TABLE IF NOT EXISTS players (
    player_id SERIAL,
    player_fide_id INT,
    player_name VARCHAR(256),
    PRIMARY KEY(player_id)
)
"""

create_table_game_details = """
CREATE TABLE IF NOT EXISTS games (
    game_id SERIAL,
    white_id INT,
    black_id INT,
    white_elo INT,
    black_elo INT,
    white_title VARCHAR(16),
    black_title VARCHAR(16),
    event VARCHAR(256),
    site VARCHAR(256),
    date VARCHAR(64),
    round VARCHAR(64),
    result VARCHAR(16),
    eco VARCHAR(16),
    opening VARCHAR(256),
    variation VARCHAR(256),
    PRIMARY KEY(game_id)
)
"""

cur.execute("DROP TABLE IF EXISTS positions")
cur.execute("DROP TABLE IF EXISTS players")
cur.execute("DROP TABLE IF EXISTS games")
cur.execute(create_table_positions)
cur.execute(create_table_players)
cur.execute(create_table_game_details)

In [6]:
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
"""

cur.execute(query)
tables = cur.fetchall()
print("All tables in the database:")
for table in tables:
    print(table[0])

All tables in the database:
positions
players
games


In [7]:
def insert_or_get_player_id(player_details):
    cur.execute("SELECT player_id FROM players WHERE player_fide_id = %s OR player_name = %s", 
                (player_details['fide_id'], player_details['name']))
    result = cur.fetchone()
    if result:
        return result[0]
    else:
        cur.execute("INSERT INTO players (player_fide_id, player_name) VALUES (%s, %s) RETURNING player_id", 
                    (player_details['fide_id'], player_details['name']))
        conn.commit()
        return cur.fetchone()[0]

for game_count, game in enumerate(games):
    metadata_dict = dict(game.headers)
    game_matrix = game_to_csr_matrix(game, csr=False)

    cur.execute("""
    INSERT INTO games (event, site, date, round, result, eco, opening, variation, white_elo, black_elo, white_title, black_title) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) RETURNING game_id
    """, (
        metadata_dict.get('Event'), metadata_dict.get('Site'), metadata_dict.get('Date'),
        metadata_dict.get('Round'), metadata_dict.get('Result'), metadata_dict.get('ECO'),
        metadata_dict.get('Opening'), metadata_dict.get('Variation'), metadata_dict.get('WhiteElo'),
        metadata_dict.get('BlackElo'), metadata_dict.get('WhiteTitle'), metadata_dict.get('BlackTitle')
    ))
    game_id = cur.fetchone()[0]
    
    white_id = insert_or_get_player_id({'fide_id': metadata_dict.get('WhiteFideId'), 'name': metadata_dict['White']})
    black_id = insert_or_get_player_id({'fide_id': metadata_dict.get('BlackFideId'), 'name': metadata_dict['Black']})
    cur.execute("UPDATE games SET white_id = %s, black_id = %s WHERE game_id = %s", (white_id, black_id, game_id))

    integer_matrix = game_matrix[:, :780]
    float_matrix = game_matrix[:, 780:]
    
    position_records = []
    for i in range(len(game_matrix)):
        position_vars = get_keys_func(np.concatenate([integer_matrix[i], float_matrix[i]]))
        position_record = (game_id, *position_vars, list(integer_matrix[i]), list(float_matrix[i]))
        position_records.append(position_record)

    execute_values(cur, """
    INSERT INTO positions (game_id, self_material_count, opp_material_count, diff_material_count, 
    self_castle_available, opp_castle_available, position_in_endgame, vector_int, vector_float) 
    VALUES %s
    """, position_records)
    if game_count % 64 == 0:
        conn.commit()
conn.commit()

In [8]:
query = """
WITH combined_ids AS (
    SELECT black_id AS player_id FROM games
    UNION ALL
    SELECT white_id AS player_id FROM games
),
frequency AS (
    SELECT player_id, COUNT(*) AS freq
    FROM combined_ids
    GROUP BY player_id
)
SELECT p.player_name, f.freq
FROM frequency f
JOIN players p ON p.player_id = f.player_id
ORDER BY f.freq DESC LIMIT 100;
"""

#self_material_count, opp_material_count, diff_material_count, self_castle_available, opp_castle_available, position_in_endgame
sm, om, dm, sc, oc, pe = (10,10,0,0,0,1)

query = f'''
SELECT COUNT(*) FROM positions
WHERE self_material_count < {sm+5.5} AND self_material_count > {sm-5.5} 
AND self_material_count < {om+5.5} AND self_material_count > {om-5.5} 
AND self_material_count < {dm+5.5} AND self_material_count > {dm-5.5} 
AND (self_castle_available = {sc} OR opp_castle_available = {oc})
AND position_in_endgame = {pe}
'''

cur.execute(query)
tables = cur.fetchall()
print("All tables in the database:")
for table in tables:
    print(table)

All tables in the database:
(6575,)
